In [3]:
import torch

import cv2

from DexiNed import DexiNed

import numpy as np

dir_weights = '/home/gzguevara/ai_laser/engines/10_model.pth'

# Load Model

In [4]:
model = DexiNed().to('cuda')

In [5]:
model.load_state_dict(torch.load(f=dir_weights, map_location='cuda'))

<All keys matched successfully>

# Build ONNX File

In [6]:
#Define batch size and frame shape
input_tensor = torch.rand(1, 3, 480, 640, device='cuda')

dir_onnx = "/home/gzguevara/laser/engines/dexi_block4_640x480.onnx"

torch.onnx.export(model,                              # model being run
                  input_tensor,                       # model input (or a tuple for multiple inputs)
                  dir_onnx,                           # where to save the model (can be a file or file-like object)
                  export_params       = True,         # store the trained parameter weights inside the model file
                  opset_version       = 12,           # the ONNX version to export the model to
                  do_constant_folding = True,         # whether to execute constant folding for optimization
                  input_names         = ['input'],    # the model's input names
                  output_names        = ['output']    # the model's output names
)

====== Diagnostic Run torch.onnx.export version 2.0.0a0+8aa34602.nv23.03 =======
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



# ONNX to TRT

In [1]:
!/home/gzguevara/bin/trtexec --onnx=/home/gzguevara/laser/engines/dexi_block4_640x480.onnx \
                             --saveEngine=/home/gzguevara/laser/engines/dexi_block4_640x480_fp16.trt \
                             --fp16
                        

&&&& RUNNING TensorRT.trtexec [TensorRT v8401] # /home/gzguevara/bin/trtexec --onnx=/home/gzguevara/laser/engines/dexi_block4_640x480.onnx --saveEngine=/home/gzguevara/laser/engines/dexi_block4_640x480_fp16.trt --fp16
[05/19/2023-20:34:05] [I] === Model Options ===
[05/19/2023-20:34:05] [I] Format: ONNX
[05/19/2023-20:34:05] [I] Model: /home/gzguevara/laser/engines/dexi_block4_640x480.onnx
[05/19/2023-20:34:05] [I] Output:
[05/19/2023-20:34:05] [I] === Build Options ===
[05/19/2023-20:34:05] [I] Max batch: explicit batch
[05/19/2023-20:34:05] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[05/19/2023-20:34:05] [I] minTiming: 1
[05/19/2023-20:34:05] [I] avgTiming: 8
[05/19/2023-20:34:05] [I] Precision: FP32+FP16
[05/19/2023-20:34:05] [I] LayerPrecisions: 
[05/19/2023-20:34:05] [I] Calibration: 
[05/19/2023-20:34:05] [I] Refit: Disabled
[05/19/2023-20:34:05] [I] Sparsity: Disabled
[05/19/2023-20:34:05] [I] Safe mode: Disabled
[05/19/

In [ ]:
36.9378 allblocks
52.6109 block4
122.402 91.0887 block4 fp16
206.886 block4 int8 -- results in broken image. maybe calibration needed

# Inspect Image Quality 

In [11]:
def image_normalization(img, img_min=0, img_max=255):
    img     = np.float32(img)
    epsilon = 1e-12  # whenever an inconsistent image
    img     = (img-np.min(img))*(img_max-img_min) / ((np.max(img)-np.min(img))+epsilon)+img_min
    return img

In [ ]:
input_file  = '/home/gzguevara/laser/media/input/input_girl.mp4'
output_file = '/home/gzguevara/laser/media/output/output_girl.mp4'

cap = cv2.VideoCapture(input_file)

# Get the video properties
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out    = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

while cap.isOpened():

    ret, frame = cap.read()

    if not ret: break

    frame  = np.array(frame, dtype=np.float32)
    frame -= [103.939, 116.779, 123.68] #Mean subtraction
    frame  = frame.transpose((2, 0, 1)) #Move chanels to the last index 
    frame  = torch.from_numpy(frame).float().cuda()
    frame  = frame.unsqueeze(dim=0)

    with torch.no_grad():
        frame = model(frame)
        torch.cuda.empty_cache()
    
    frame = torch.squeeze(frame) #(1,1,512,512) -> (512,512)
    frame = torch.sigmoid(frame).cpu().detach().numpy() 
    frame = frame * 255 #image_normalization(frame) 
    frame = frame.astype(np.uint8)
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

    out.write(frame)

cap.release()
out.release()